In [20]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [21]:
# get data in desirved format
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "highest_price": "float",
    "lowest_price": "float", 
    "close_price": "float",
    "volumn": "int"}
print("Start read dow file...")
for dow_name in dow_files_names:
    df = pd.read_csv(dow_files_addr + dow_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    df = df.sort_values(by=["datetime"], ascending=False)
    dow_dfs.append(df)
print("Finish read dow file.")
print("---------------------")
print("Start read etf file...")
for etf_name in etf_files_names:
    df = pd.read_csv(etf_files_addr + etf_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    df = df.sort_values(by=["datetime"], ascending=False)
    etf_dfs.append(df)
print("Finish read etf file.")

Start read dow file...
Finish read dow file.
---------------------
Start read etf file...
Finish read etf file.


In [22]:
# get start and end for both dow and etf that cove evey df
dow_time_slice = [min(dow_dfs[0]["datetime"]), max(dow_dfs[0]["datetime"])]
etf_time_slice = [min(etf_dfs[0]["datetime"]), max(etf_dfs[0]["datetime"])]
print("Start check dow date...")
i = 0
for df in dow_dfs:
    start = min(df["datetime"])
    if start > dow_time_slice[0]:
        print("start", dow_files_names[i])
        print(start)
        dow_time_slice[0] = start
    end = max(df["datetime"])
    if end < dow_time_slice[1]:
        print("end", dow_files_names[i])
        print(end)
        dow_time_slice[1] = end
    i += 1
print("Finish check dow date...")
print("---------------------")
print("Start check etf date...")
i = 0
for df in etf_dfs:
    start = min(df["datetime"])
    if start > etf_time_slice[0]:
        print("start", dow_files_names[i])
        print(start)
        etf_time_slice[0] = start
    end = max(df["datetime"])
    if end < etf_time_slice[1]:
        print("end", dow_files_names[i])
        print(end)
        etf_time_slice[1] = end
    i += 1
print("Finish check etf date...")

Start check dow date...
start GM.txt
2010-11-18 09:36:00
end GM.txt
2020-06-05 19:44:00
Finish check dow date...
---------------------
Start check etf date...
start AAPL.txt
2003-04-14 09:32:00
end AAPL.txt
2020-06-05 19:41:00
end AIG.txt
2020-06-05 17:38:00
start AXP.txt
2008-11-19 09:30:00
start T.txt
2009-06-25 09:40:00
Finish check etf date...


In [24]:
dow_time_slice

[Timestamp('2010-11-18 09:36:00'), Timestamp('2020-06-05 19:44:00')]

In [25]:
etf_time_slice

[Timestamp('2009-06-25 09:40:00'), Timestamp('2020-06-05 17:38:00')]

In [19]:
dow_dfs[0]

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime
1470600,06/05/2020,19:59,12.00,12.00,12.00,12.00,1800,2020-06-05 19:59:00
1470599,06/05/2020,19:56,12.00,12.00,12.00,12.00,400,2020-06-05 19:56:00
1470598,06/05/2020,19:55,12.04,12.04,12.04,12.04,1791,2020-06-05 19:55:00
1470597,06/05/2020,19:52,12.05,12.05,12.05,12.05,416,2020-06-05 19:52:00
1470596,06/05/2020,19:44,12.05,12.05,12.05,12.05,384,2020-06-05 19:44:00
...,...,...,...,...,...,...,...,...
4,04/27/2007,10:56,73.80,73.82,73.78,73.82,5324,2007-04-27 10:56:00
3,04/27/2007,10:55,73.78,73.80,73.78,73.80,2904,2007-04-27 10:55:00
2,04/27/2007,10:54,73.78,73.80,73.78,73.78,3340,2007-04-27 10:54:00
1,04/27/2007,10:53,73.74,73.80,73.74,73.80,2323,2007-04-27 10:53:00


In [13]:
tmp = tmp.sort_values(by=["datetime"], ascending=False)

In [14]:
tmp

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime
20861,07/16/2007,09:40,101.230,101.270,100.6900,100.6900,74693,2007-07-16 09:40:00
20860,07/16/2007,09:39,101.210,101.340,100.8800,101.2500,90190,2007-07-16 09:39:00
20862,07/16/2007,09:41,100.690,100.710,100.2600,100.4400,82009,2007-07-16 09:41:00
20856,07/16/2007,09:35,100.550,100.650,99.9600,100.0700,104821,2007-07-16 09:35:00
20863,07/16/2007,09:42,100.440,100.570,100.2600,100.4400,66656,2007-07-16 09:42:00
...,...,...,...,...,...,...,...,...
1445322,03/19/2020,09:43,5.235,5.274,5.2333,5.2650,10130,2020-03-19 09:43:00
1446219,03/23/2020,09:40,5.230,5.300,5.1600,5.2000,28845,2020-03-23 09:40:00
1445320,03/19/2020,09:41,5.220,5.230,5.2000,5.2173,19012,2020-03-19 09:41:00
1446220,03/23/2020,09:41,5.210,5.240,5.1900,5.2400,15014,2020-03-23 09:41:00


In [37]:
1. test sort_value
2. check minute assertion

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime
0,04/27/2007,10:52,73.72,73.74,73.72,73.74,7405,2007-04-27 10:52:00
1,04/27/2007,10:53,73.74,73.80,73.74,73.80,2323,2007-04-27 10:53:00
2,04/27/2007,10:54,73.78,73.80,73.78,73.78,3340,2007-04-27 10:54:00
3,04/27/2007,10:55,73.78,73.80,73.78,73.80,2904,2007-04-27 10:55:00
4,04/27/2007,10:56,73.80,73.82,73.78,73.82,5324,2007-04-27 10:56:00
...,...,...,...,...,...,...,...,...
1470596,06/05/2020,19:44,12.05,12.05,12.05,12.05,384,2020-06-05 19:44:00
1470597,06/05/2020,19:52,12.05,12.05,12.05,12.05,416,2020-06-05 19:52:00
1470598,06/05/2020,19:55,12.04,12.04,12.04,12.04,1791,2020-06-05 19:55:00
1470599,06/05/2020,19:56,12.00,12.00,12.00,12.00,400,2020-06-05 19:56:00


In [38]:
min(tmp["datetime"])

Timestamp('2007-04-27 10:52:00')

In [39]:
max(tmp["datetime"])

Timestamp('2020-06-05 19:59:00')